In [17]:
!pip install xmltodict

In [8]:
!pip install opencv-python

     ---------------------------------------- 38.2/38.2 MB 8.6 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
tf.__version__

'2.10.1'

In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import load_img, img_to_array

from PIL import Image
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import cv2
import xmltodict
import json

In [3]:
pd.read_xml('../Semi_Data/annotations/BikesHelmets0.xml')

,folder,filename,width,height,depth,segmented,name,pose,truncated,occluded,difficult,bndbox
0,images,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN
1,None,BikesHelmets0.png,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN
2,None,None,400.0,267.0,3.0,NaN,None,None,NaN,NaN,NaN,NaN
3,None,None,NaN,NaN,NaN,0.0,None,None,NaN,NaN,NaN,NaN
4,None,None,NaN,NaN,NaN,NaN,With Helmet,Unspecified,0.0,0.0,0.0,NaN
5,None,None,NaN,NaN,NaN,NaN,With Helmet,Unspecified,0.0,0.0,0.0,NaN
6,None,None,NaN,NaN,NaN,NaN,With Helmet,Unspecified,0.0,0.0,0.0,NaN
7,None,None,NaN,NaN,NaN,NaN,With Helmet,Unspecified,0.0,0.0,0.0,NaN


In [7]:
pip install xmltodict

  Using cached xmltodict-0.13.0-py2.py3-none-any.whl (10.0 kB)
Note: you may need to restart the kernel to use updated packages.


In [14]:
# import json
# import xmltodict

# ## xml 파일을 json 파일로 변환

# for i in range(766) :
#     try :
#         with open(f"../Semi_Data/annotations/BikesHelmets{i}.xml",'r') as f:
#             xmlString = f.read()
            
#         jsonString = json.dumps(xmltodict.parse(xmlString), indent=4)
        
#         with open(f"../Semi_Data/annotation/BikesHelmets{i}.json", 'w') as f:
#             f.write(jsonString)
#     except :
#         pass

In [10]:
import cv2

# 이미지 로드
image = cv2.imread('../Semi_Data/images/BikesHelmets0.png')
# image.shape
# 이미지 크기 조정
image = cv2.resize(image, (416, 416))

cv2.imwrite('./practice.png', image)
# 픽셀 값 정규화
image = image.astype('float32')/255.0

# YOLO 모델 입력 데이터 형식으로 변환
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = np.expand_dims(image, axis = 0) # 이미지 배열의 차원을 하나 추가해서 (1, height, width, channels) 형식으로 변환

(267, 400, 3)

In [43]:
import cv2
import json

for i in range(766) :
    try :
        # json 파일에서 bndbox 정보 추출
        with open(f'../Semi_Data/annotation/BikesHelmets{i}.json', 'r') as f:
            label = json.load(f)

        bndboxes = []
        for obj in label['annotation']['object']:
            bndbox = obj['bndbox']
            xmin = int(bndbox['xmin'])
            ymin = int(bndbox['ymin'])
            xmax = int(bndbox['xmax'])
            ymax = int(bndbox['ymax'])
            bndboxes.append((xmin, ymin, xmax, ymax))

        # 이미지 로드
        img = cv2.imread(f'../Semi_Data/images/BikesHelmets{i}.png')

        # 경계 상자 그리기
        for bndbox in bndboxes:
            cv2.rectangle(img, (bndbox[0], bndbox[1]), (bndbox[2], bndbox[3]), (0, 255, 0), 2)

        # 이미지 저장
        cv2.imwrite(f'../Semi_Data/bounded_images/BikeHelmets{i}.png', img) # 저장할 경로 입력
    except :
        try :
            with open(f'../Semi_Data/annotation/BikesHelmets{i}.json', 'r') as f:
                label = json.load(f)
            
            bndboxes = []
            bndbox = label['annotation']['object']['bndbox']
            xmin = int(bndbox['xmin'])
            ymin = int(bndbox['ymin'])
            xmax = int(bndbox['xmax'])
            ymax = int(bndbox['ymax'])
            bndboxes.append((xmin, ymin, xmax, ymax))
            
            img = cv2.imread(f'../Semi_Data/images/BikesHelmets{i}.png')

        # 경계 상자 그리기
            for bndbox in bndboxes:
                cv2.rectangle(img, (bndbox[0], bndbox[1]), (bndbox[2], bndbox[3]), (0, 255, 0), 2)

            # 이미지 저장
            cv2.imwrite(f'../Semi_Data/bounded_images/BikeHelmets{i}.png', img) # 저장할 경로 입력
            
        except :
            pass
        


In [17]:
def save_data(file_path, image_save_path, label_save_path, save_file_index) :
    
    idx = save_file_index
    
    # file path에 있는 파일들 리스트로 담기
    file_list = os.listdir(file_path)
    
    # 파일이름에 .png가 있으면 image_list에 담기
    image_list = [i for i in file_list if '.png' in i]
    
    # 파일이름에 .xml이 있으면 label_list에 담기
    label_list = [i for i in file_list if '.xml' in i]
    
    
    
    for image,label in zip(image_list, label_list) : 
        # image 로드
        img = cv2.imread(file_path + image)
        
        # 이미지 저장
        cv2.imwrite(image_save_path + f'helmet_img_{idx}.png', img) # 저장할 경로 입력
        
        # xml 파일 json으로 바꿔서 저장하기    
    
        with open(file_path + label,'r',encoding='UTF8') as f:
            xmlString = f.read()
            
        jsonString = json.dumps(xmltodict.parse(xmlString), indent=4)
        
        with open(label_save_path + f'helmet_label_{idx}.json', 'w') as f:
            f.write(jsonString)
    
        idx += 1
    
    print(f'마지막 파일 idx는 {idx - 1}입니다.')
   

In [26]:
# save_data('./Data/helmet datas/With Helmet/','./Data/helmet_image/', './Data/helmet_label/', 0)

마지막 파일 idx는 99입니다.


In [27]:
# save_data('./Data/helmet datas/Without Helmet/','./Data/helmet_image/', './Data/helmet_label/', 100)

마지막 파일 idx는 199입니다.


In [47]:
# save_data('./Data/helmet_YB/With_Helmet_image/','./Data/helmet_image/', './Data/helmet_label/', 200)

마지막 파일 idx는 299입니다.


In [48]:
# save_data('./Data/helmet_YB/Without_Helmet_image/','./Data/helmet_image/', './Data/helmet_label/', 300)

마지막 파일 idx는 399입니다.


In [49]:
# save_data('./Data/kaggle_image_resize/','./Data/helmet_image/', './Data/helmet_label/', 400)

마지막 파일 idx는 1123입니다.


In [50]:
# save_data('./Data/withouthelmet_resize/','./Data/helmet_image/', './Data/helmet_label/', 1124)

마지막 파일 idx는 1326입니다.


In [18]:
save_data('./data/without_helmet_add_resize/','./data/helmet_image/', './data/helmet_label/', 1327)

마지막 파일 idx는 1783입니다.


In [20]:
# 이미지를 자르고 저장하는 함수
def cut_image(json_path, image_path) : 
    
    with_idx = 0
    without_idx = 0
    json_file = os.listdir(json_path)
    image_file = os.listdir(image_path)
    n = len(image_file)
    
    for i in range(n) :
        try :
            # json 파일에서 bndbox 정보 추출
            with open(json_path + json_file[i], 'r') as f:
                label = json.load(f)
            
            bndboxes = []
                            
            for obj in label['annotation']['object']:
                bndbox = obj['bndbox']
                img_l = obj['name']
                xmin = int(bndbox['xmin'])
                ymin = int(bndbox['ymin'])
                xmax = int(bndbox['xmax'])
                ymax = int(bndbox['ymax'])
                bndboxes.append((xmin, ymin, xmax, ymax, img_l))
            # print('clear')
            # 이미지 자르기
            for bndbox in bndboxes:
                try : 
                    # 이미지 로드
                    img = Image.open(image_path + image_file[i])
                    image = img.crop((bndbox[0], bndbox[1], bndbox[2], bndbox[3]))
                    # 이미지 저장
                    if bndbox[4] == 'With Helmet' :
                        image.save(f'./data/with_helmet/with_helmet_{with_idx}.png') # 저장할 경로 입력
                        with_idx += 1
                    
                    else :
                        image.save(f'./data/without_helmet/without_helmet_{without_idx}.png')
                        without_idx += 1
                except :
                    print(json_file[i])
        except :
            try :
                with open(json_path + json_file[i], 'r') as f:
                    label = json.load(f)
                
                bndbox = label['annotation']['object']['bndbox']
                img_l = label['annotation']['object']['name']
                xmin = int(bndbox['xmin'])
                ymin = int(bndbox['ymin'])
                xmax = int(bndbox['xmax'])
                ymax = int(bndbox['ymax'])
            
                
                # 이미지 자르기
                
                # 이미지 로드
                img = Image.open(image_path + image_file[i])
                image = img.crop((xmin, ymin, xmax, ymax))
                
                if img_l == 'With Helmet' :
                    image.save(f'./data/with_helmet/with_helmet_{with_idx}.png') # 저장할 경로 입력
                    with_idx += 1
                
                else :
                    image.save(f'./data/without_helmet/without_helmet_{without_idx}.png')
                    without_idx += 1
            except :
                pass

    

In [21]:
image_label = cut_image('./data/helmet_label/', './data/helmet_image/')

In [ ]:
fiel_list = os.listdir('./Data/cut_image/')

In [3]:
# 이미지 사이즈 조절하여 저장 하는 함수

def img_resize_save(img_path, save_path, image_shape) :
    img_list = os.listdir(img_path)
    count = 0
    
    if not os.path.exists(save_path) :
        os.mkdir(save_path)
        
    for img in img_list :
        image = cv2.imread(img_path + img)
        
        image = cv2.resize(image, image_shape)
        
        cv2.imwrite(save_path+img, image)
        count += 1


In [6]:
img_resize_save('./data/with_helmet_resize_128/','./data/with_helmet_resize_64/',(64, 64))

In [7]:
img_resize_save('./data/without_helmet_resize_128/','./data/without_helmet_resize_64/',(64, 64))

In [59]:
img_resize_save('./Data/cut_image/', './Data/cut_image_resize/')


In [13]:
img_resize_save('./data/with_helmet_gen/', './data/with_helmet_resize_128/', (128, 128))

In [24]:
img_resize_save('./data/without_helmet_add_gen/', './data/without_helmet_resize_128/', (128, 128))

In [61]:
file_list = os.listdir('./Data/cut_image_resize/')
df = pd.DataFrame([file_list, image_label])

In [63]:
df = df.T

In [65]:
df.columns = ['file_name', 'label']
df

,file_name,label
0,cut_helmet_0.png,With Helmet
1,cut_helmet_1.png,With Helmet
2,cut_helmet_10.png,With Helmet
3,cut_helmet_100.png,Without Helmet
4,cut_helmet_1000.png,Without Helmet
...,...,...
2064,cut_helmet_995.png,Without Helmet
2065,cut_helmet_996.png,Without Helmet
2066,cut_helmet_997.png,Without Helmet
2067,cut_helmet_998.png,With Helmet


In [66]:
df.to_csv('./label.csv', index = False)

In [2]:
def BW_image(img_path, save_path) :
    file_list = os.listdir(img_path)
    for img in file_list :
        color_image = Image.open(img_path + img)
        
        gray_image = color_image.convert('L')
        
        gray_image.save(save_path + img)

In [3]:
BW_image('./data/with_helmet_resize/', './data/with_helmet_BW/')

In [4]:
BW_image('./data/without_helmet_resize/', './data/without_helmet_BW/')